# TTN遺伝子の両端の配列に着目しmotif配列の検出を行う

## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/human_ttn.gb')

In [ ]:
gbk.set_interest_seq("TCTT")

In [ ]:
# 各バリアントのエクソン数を取得
gbk.variant_exons()

In [ ]:
motif
def check_dist(motif):
    

In [57]:
motif = "ACTAATATCTT"
all_intron = ""
for i in range(gbk.intron_num()):
    all_intron += gbk.intron_seq(i+1)

motif_list = [0]*(len(all_intron)-len(motif))
for i in range(len(all_intron)-len(motif)):
    if all_intron[i:i+len(motif)] == motif:
        motif_list[i] = 1

# motifをn個のサブリストに分割
n = 20
length = len(motif_list)
base = length // n
remainder = length % n

# base個の要素を持つn個のリストと、余りを配分
split_lists = [motif_list[i * base + min(i, remainder):(i + 1) * base + min(i + 1, remainder)] for i in range(n)]

one_counts = []
for i in split_lists:
    one_counts.append(i.count(1))

# 合計を100として標準偏差を求める
arr = np.array(one_counts)*(100/np.array(one_counts).sum())
arr.std()

14.023385069168782

In [55]:
one_counts

[3, 2, 3, 8, 6, 3, 2, 3, 8, 5, 6, 3, 7, 3, 2, 2, 6, 3, 4, 3]

In [51]:
# 各イントロンの両端n塩基を抽出し
n = 30

# 各イントロンの5-prime側のn塩基を抽出 
left_egdes = []
for i in range(gbk.intron_num()):
    left_egdes.append(str(gbk.intron_seq(i+1)[:n]))
    
# 各イントロンの3-prime側のn塩基を抽出
right_egdes = []
for i in range(gbk.intron_num()):
    right_egdes.append(str(gbk.intron_seq(i+1)[-n:]))

### 抽出した配列のワンホットエンコーディングと主成分分析
各イントロンの5-prime側のn塩基の主成分分析

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences = np.array([one_hot_encode(seq) for seq in left_egdes])

# 主成分分析（PCA）
pca_l = PCA(n_components=2)  # 2つの主成分を取得
principal_components = pca_l.fit_transform(encoded_sequences)

# 主成分をデータフレームに変換
df_pca_l = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 結果をプロット
fig = px.scatter(df_pca_l, x='PC1', y='PC2', hover_name=[f"intron_{i+1}" for i in range(gbk.intron_num())] )
fig.show()


In [ ]:
# 各イントロンのPC1の値を棒グラフにする
fig = px.bar(x=df_pca_l.index, y=df_pca_l['PC1'])
fig.show()

In [ ]:
# 各バリアントのエクソンを表示
gbk.transcript_variants()

In [ ]:
# 因子負荷量の計算
loadings_l = pca_l.components_.T * np.sqrt(pca_l.explained_variance_)

# 因子負荷量を縦軸に棒グラフを作成
loadings_ldf = pd.DataFrame(loadings_l, columns=['PC1', 'PC2'])
ldf_l = pd.DataFrame(loadings_ldf)
ldf_l.sort_values('PC1')
fig = px.bar(x=ldf_l.index, y=ldf_l['PC1'])
fig.show()

In [ ]:
# ワンホットエンコーディングをデコードする
# 負荷量の最も高い塩基を取得し、配列を作成する
b_df_l = pd.DataFrame(ldf_l['PC1'].to_numpy().reshape(n,4))
b_df_l.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(b_df_l, shade_below=.5, fade_below=.5)

各イントロンの3-prime側のn塩基の主成分分析

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences = np.array([one_hot_encode(seq) for seq in right_egdes])

# 主成分分析（PCA）
pca_r = PCA(n_components=2)  # 2つの主成分を取得
principal_components = pca_r.fit_transform(encoded_sequences)

# 主成分をデータフレームに変換
df_pca_r = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 結果をプロット
fig = px.scatter(df_pca_r, x='PC1', y='PC2', hover_name=[f"intron_{i+1}" for i in range(gbk.intron_num())] )
fig.show()

In [ ]:
# 各イントロンのPC1の値を棒グラフにする
fig = px.bar(x=df_pca_r.index, y=df_pca_r['PC1'])
fig.show()

In [ ]:
# 各バリアントのエクソンを表示
gbk.transcript_variants()

In [ ]:
# 因子負荷量の計算
loadings_r = pca_r.components_.T * np.sqrt(pca_r.explained_variance_)

# 因子負荷量を縦軸に棒グラフを作成
loadings_rdf = pd.DataFrame(loadings_r, columns=['PC1', 'PC2'])
ldf_r = pd.DataFrame(loadings_rdf)
ldf_r.sort_values('PC1')
fig = px.bar(x=ldf_r.index, y=ldf_r['PC1'])
fig.show()

In [ ]:
# ワンホットエンコーディングをデコードする
# 負荷量の最も高い塩基を取得し、配列を作成する
b_df_r = pd.DataFrame(ldf_r['PC1'].to_numpy().reshape(n,4))
b_df_r.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(b_df_r, shade_below=.5, fade_below=.5)

In [ ]:
# 上記の配列の一部が末尾n塩基に含まれているイントロンを抽出
mtf = "ACTAATATCTT"
for i in range(gbk.intron_num()):
    if mtf in gbk.intron_seq(i+1)[-n:]:
        print(f"intron_{i+1}の末尾{n}塩基")
        seq = str(gbk.intron_seq(i+1)[-n:])
        seq = seq.replace(mtf,f"<{mtf}>")
        print(seq)
        print("")

In [ ]:
# 上記の配列の一部を含むイントロンの部位を可視化
gbk.set_interest_seq('ACTAATATCTT')
gbk.heatmap_hist()

## Proporton Spliced In
参考論文 [Contribution of Noncanonical Splice Variants to TTN Truncating Variant Cardiomyopathy](https://www.ahajournals.org/doi/full/10.1161/CIRCGEN.121.003389)<br>
<br>
**Figure 5A:**<br>
[GTEX](https://gtexportal.org/home/)データベース情報をもとに骨格筋、左心室心筋、右心房心筋のmRNAのスプライスインされたエクソンの割合を示すグラフ

![PSI](../data/pic/Figure5A.png)